# Sentiment Analysis sulle Recensioni di Yelp

La Sentiment Analysis è il processo di identificazione dell'emozione espressa in un testo, positiva o negativa.

In questo notebook useremo Spark e la sua MLlib per costruire un modello di Sentiment Analysis usando il dataset messo a disposizione da Yelp, una famossisima applicazione che permette di recensire locali e attività commerciali.


# Inizializziamo Spark

In [17]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("sentiment_analysis").setMaster("local").set("spark.driver.memory", "5g")
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)

# Importiamo il dataset in un dataframe

In [18]:
yelp_df = sqlContext.read.json('./data/yelp_academic_dataset_review.json')

In [19]:
yelp_df.count()

6685900

In [20]:
yelp_df.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [21]:
reviews_df = yelp_df.select(["text", "stars"])

In [23]:
subreviews_df = reviews_df.sample(False, 0.01, seed=0)

In [24]:
subreviews_df.count()

67136

# Preprocessing del testo

In [26]:
import string

def remove_punct(text):
    return text.translate(str.maketrans('','', string.punctuation))

remove_punct(".... che cacchio dici, Antonio !!!1!")

' che cacchio dici Antonio 1'

In [28]:
from pyspark.sql.functions import udf

punct_remove = udf(lambda s: remove_punct(s))

In [30]:
subreviews_df = subreviews_df.withColumn("text", punct_remove(reviews_df["text"]))

In [31]:
subreviews_df.show(20)

+--------------------+-----+
|                text|stars|
+--------------------+-----+
|So many great thi...|  5.0|
|If I could give t...|  1.0|
|Lovely food I am ...|  5.0|
|Weve always been ...|  4.0|
|After agonizing o...|  4.0|
|Even though a few...|  4.0|
|No stars yes no s...|  1.0|
|Not bad at all  T...|  3.0|
|So arrived at 530...|  1.0|
|Excellent Excelle...|  5.0|
|We go to Vegas at...|  5.0|
|Good flavor fast ...|  5.0|
|First time here a...|  5.0|
|I was shocked tha...|  1.0|
|Large portions Gr...|  5.0|
|This is a great l...|  5.0|
|Always good  One ...|  5.0|
|Wonderful service...|  4.0|
|On the weeklong t...|  3.0|
|Awesome burgers g...|  5.0|
+--------------------+-----+
only showing top 20 rows

